In [16]:
import pandas as pd

PATH = '/Users/danielsagher/Dropbox/Documents/projects/bls_api_project'

national_results = pd.read_csv('../api_call/main_output/national_results.csv')
state_results = pd.read_csv('../api_call/main_output/state_results.csv')
national_series_dimension = pd.read_csv('../csv_from_excel/national_series_dimension.csv')
state_series_dimension = pd.read_csv('../state_scrape/output/state_series_dimension.csv')
survey_table = pd.read_csv('../csv_from_excel/survey_table.csv')



In [17]:
pd.merge(national_results[(national_results.duplicated() == True)], national_series_dimension, how='left')\
        .series.unique()

array([], dtype=object)

In [18]:
national_results = national_results.drop_duplicates()
national_results.duplicated().sum()

0

In [19]:
national_results.dtypes

seriesID       object
year            int64
period         object
period_name    object
value          object
footnotes      object
dtype: object

In [20]:
national_results['value'] = national_results['value'].replace('-', 0)
national_results['value'] = national_results['value'].astype('float')

national_results.dtypes

seriesID        object
year             int64
period          object
period_name     object
value          float64
footnotes       object
dtype: object

In [21]:

def space_remover(df):

        for col in df.columns:

                df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
                
        return df


national_results = space_remover(national_results)
national_series_dimension = space_remover(national_series_dimension)
state_results = space_remover(state_results)
state_series_dimension = space_remover(state_series_dimension)
survey_table = space_remover(survey_table)
                

In [22]:
national_results.footnotes = [i.replace('[{}]', 'NaN') for i in national_results.footnotes]
state_results.footnotes = [i.replace('[{}]', 'NaN') for i in state_results.footnotes]


In [23]:
national_series_dimension['is_adjusted'] = \
    national_series_dimension['series'].str.lower().apply(lambda x: False if 'not seasonally adjusted' in x else True)

state_series_dimension['is_adjusted'] = \
   state_series_dimension['series'].str.lower().apply(lambda x: False if 'not seasonally adjusted' in x else True)


In [24]:
state_series_dimension.series[(state_series_dimension.is_adjusted == False)] 
national_series_dimension.series[(national_series_dimension.is_adjusted == False)]

Series([], Name: series, dtype: object)

In [25]:
# national_series_dimension.series = national_series_dimension['series'].str.lower().replace('seasonally adjusted', '').replace('not', '')
# state_series_dimension['series'] = state_series_dimension['series'].str.replace('Seasonally adjusted', '').replace('Not', '')

def adjusted_remover(series):
    term1 = ', seasonally adjusted'
    term2 = ', not seasonally adjusted'
    
    def remove_terms(text):
        text_lower = text.lower()
        index1 = text_lower.find(term1)
        index2 = text_lower.find(term2)
        if index1 != -1:
            return text[:index1]
        elif index2 != -1:
            return text[:index2]
        return text
    
    # Apply the removal function to the 'series' column
    series['series'] = series['series'].apply(remove_terms)
    return series

# adjusted_remover(national_series_dimension)
national_series_dimension = adjusted_remover(national_series_dimension)
state_series_dimension = adjusted_remover(state_series_dimension)



In [26]:
national_series_dimension[(national_series_dimension['series'].str.contains('seasonally adjusted') == True)].sum(),\
state_series_dimension[(state_series_dimension['series'].str.contains('seasonally adjusted') == True)].sum()

(series         0
 seriesID       0
 survey         0
 is_adjusted    0
 dtype: object,
 series         0
 seriesID       0
 state          0
 is_adjusted    0
 survey         0
 dtype: object)

In [27]:
national_series_dimension.to_csv('../cleaning/output/national_seriesID_list_cleaned.csv', index=False)
state_series_dimension.to_csv('../cleaning/output/state_series_dimension.csv_cleaned.csv', index=False)